<a href="https://colab.research.google.com/github/Anish1337/TransformerCaptioner/blob/main/Captioner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U timm
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [31]:
!pip3 install torchtext


Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/parser.py", line 12, in <module>
    from pip._internal.configuration import Configuration, ConfigurationError
  File "/usr/lib/python3/dist-packages/pip/_internal/configuration.py", line 26, in <module>
    f

In [29]:
import torch
import torch.nn as nn
import albumentations as alb
import timm
import cv2
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import torchtext
from collections import Counter

from torchtext.data.utils import get_tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'torchtext'

In [16]:
class ImageCaptioner(nn.Module):
  def __init__(self, context_length, vocab_size, num_blocks, model_dim, num_heads, prob):
    super().__init__()
    self.cnn_encoder = timm.create_model('efficientnet_b0', pre_trained = True)
    test_image = torch.zeroes(1, 3, 224, 224)

    with torch.no_grad():
      cnn_output = self.cnn_encoder[test_image]
    in_features = cnn_output.shape[1]
    self.project = nn.Linear(in_features, model_dim)

    self.word_embeddings = nn.Embedding(vocab_size, model_dim)
    self.pos_embeddings = nn.Embedding(context_length, model_dim)

    block = nn.TransformerDecoderLayer(model_dim, num_heads, 2 * model_dim, dropout=prob, batch_first=True, norm_first=True)
    self.blocks = nn.TransformerDecoder(block, num_blocks)

    self.vocav_projection = nn.Linear(model_dim, vocab_size)

In [18]:
def forward(self, images, true_labels):
  tok_embedded = self.word_embeddings(true_labels)
  B, T = true_labels.shape
  positions = torch.arrange(T).to(device)
  pos_embedded = self.pos_embeddings(positions)
  total_embeddings = tok_embedded + pos_embedded

  with torch.no_grad():
    encoded_image = self.project(self.cnn_encoder(images).view(B,-1))

    img_for_attention = torch.unsqueeze(encoded_image, 1)

    attention_mask = nn.Transformer.generate_square_subsequent_mask(T).to(device)
    block_output = self.blocks(total_embeddings, img_for_attention, tgt_mask=attention_mask)\

    vocab_vector = self.vocab_projection(block_output) # B, T, V
    return vocab_vector

In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow

drive.mount('content/drive')
%cd "drive/MyDrive/Flickr8k"

caption_filename = 'captions.txt'
missing='2258277193_586949ec62.jpg' # missing image check

with open(caption_filename) as captions:
  lines = captions.readlines()

get_captions = {}
all_captions = []

for caption in lines:
  data = caption.rstrip('\n').split('.jpg,')
  img_name = data[0]+'.jpg'
  # skip missing
  if img_name == missing:
    continue

  caption_list = get_captions.get(img_name, [])
  caption_list.append(data[1])
  get_captions[img_name] = caption_list
  all_captions.append(data[1])


In [ ]:
df = pd.Dataframe(columns=['filename','caption'])
df['filename'] = get_captions.keys()
df['caption'] = df['filename'].map(lambda filename: get_captions[filename])

vocab_freq = Counter()
word_tokenizer = get_tokenizer('basic_english')